#Laboratorio 5 - IA (Redes Neuronales con PyTorch)
##Jhamil Crespo Rejas
##Ingenieria en Ciencias de la Computacion

##Importacion de Librerias

En primer lugar importamos el modulo **os** que nos permite interactuar con el sistema operativo, esto nos perimte manipular las rutas del directorio.

Necesitamos la biblioteca **numpy** para operar con los arryas y las matrices.

**torch** es una biblioteca de aprendizaje que facilita la construccion y entrenamiento de modelos de aprendizaje profundo. De esta biblioteca impotaremos varios modulos.

* **optim** que incluye varios optimizadores que utilizaremos para actualizar los parametros de la red neuronal durante el entrenamiento.
* **nn** que contiene clases para crear capas de las redes neuronales y funciones de activacion.
* **DataLoader** es una clase que automatiza la creacion de mini lotes y el muestreo de datos, lo que es esencial para el entrenamiento eficiente de modelos.

**torch.nn.functional** contiene funciones de activacion y operaciones de pooling

La biblioteca  **tqdm** nos permitira agregar una barra de progreso a los bucles.

In [1]:
# utilizado para la manipulación de directorios y rutas
import os
# Cálculo científico y vectorial para python
import numpy as np

import torch
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.

import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
from tqdm import tqdm  # For nice progress bar!

Utilizaremos **pandas** para manipular el dataset y asi poder prepararlo para el entrenamiento y prueba del modelo.

Necesitamos las funcion **train_test_split** de la bilioteca **scikit-learn** para que una vez el dataset este preparado lo dividamos en dos bloques, el primero para entrenamiento y el segundo para prueba.

In [2]:
#utilizado para importar y ordenar los datos del DATASET
import pandas as pd

from sklearn.model_selection import train_test_split


Tambien necesitamos la clase **OrdinalEncoder** del modulo **preprocessing** de la biblioteca **scikit-learn**, esta clase nos ayuda a transformar caracteristicas categoricas en un formato numerico.

In [3]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

Por ultimo, necesitamos acceder a los archivos de nuestro Google Drive, ya que dentro de este se encuentra el dataset con el que trabajaremos

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##Analisis del Dataset

El dataset contiene diferentes variables que explican los atributos de la industria de las telecomunicaciones y varios factores considerados importantes al tratar con clientes de esta industria. La variable objetivo aquí es la pérdida de clientes, que explica si el cliente abandonará o no.

En total contiene 99 caracteristicas. Pero antes de entrenar el modelo con este dataset necesitamos prepararlo.

Extraemos el dataset y lo almacenamos en un dataframe con ayuda de pandas.

In [5]:
dataframe = pd.read_csv('/content/gdrive/MyDrive/Laboratorio 05 - IA/Telecom_customer churn.csv')
print(dataframe.sample(n=200))

       rev_Mean  mou_Mean  totmrc_Mean  da_Mean  ovrmou_Mean  ovrrev_Mean  \
13559   68.6000    347.50      30.0000   0.2475        67.75      23.7125   
47970   30.1850      3.50      39.9900   0.0000         0.00       0.0000   
58941   39.8375    180.75      39.9900   0.0000        14.25       4.9875   
95921   31.0875    782.25      37.4925   0.0000         2.75       1.0950   
5222    85.0000    108.00      85.0000   0.0000         0.00       0.0000   
...         ...       ...          ...      ...          ...          ...   
4005    36.2825   1076.50      44.9900   0.4950         2.25       0.7975   
62533   48.9000    449.00      10.0000   0.9900       151.25      37.8125   
22015   90.3075    948.75      94.9900   4.4550         0.25       0.0975   
27329   87.4400    620.50      84.9900   0.0000         0.50       0.1950   
58010   58.3650    265.50      69.9900   0.0000         6.25       1.8750   

       vceovr_Mean  datovr_Mean  roam_Mean  change_mou  ...  forgntvl  ethn

###Cambio de texto a numeros

Varias de las caracteristicas son categoricas, pero para el entrenamiento y prueba del modelo necesitamos que todas las caracteristicas esten en forma numerica. Por este motivo convertiremos los datos categoricos en numericos.

####new_cell

In [6]:
new_cell_unicos = dataframe['new_cell'].unique() #Obtenemos todos los valores unicos de la columna 'new_cell'

#Mostramos la lista de valores unicos y la cantidad de estos
print(new_cell_unicos)
print(len(new_cell_unicos))

['U' 'N' 'Y']
3


In [7]:
#Se asigna valores numericos a cada uno de los trabajos
new_cell_ordinal_map = {
    'U': 1,
    'N': 2,
    'Y': 3,
}

# Se aplica el mapeo a la columna 'new_cell' y se crea una nueva columna 'new_cell_ordinal' con los valores numericos respectivos
dataframe['new_cell_ordinal'] = dataframe['new_cell'].map(new_cell_ordinal_map)

#Quitamos la columna 'new_cell'
dataframe.drop('new_cell', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   rev_Mean  mou_Mean  totmrc_Mean  da_Mean  ovrmou_Mean  ovrrev_Mean  \
0   23.9975    219.25       22.500   0.2475         0.00          0.0   
1   57.4925    482.75       37.425   0.2475        22.75          9.1   
2   16.9900     10.25       16.990   0.0000         0.00          0.0   
3   38.0000      7.50       38.000   0.0000         0.00          0.0   
4   55.2300    570.50       71.980   0.0000         0.00          0.0   

   vceovr_Mean  datovr_Mean  roam_Mean  change_mou  ...  ethnic  kid0_2  \
0          0.0          0.0        0.0     -157.25  ...       N       U   
1          9.1          0.0        0.0      532.25  ...       Z       U   
2          0.0          0.0        0.0       -4.25  ...       N       U   
3          0.0          0.0        0.0       -1.50  ...       U       Y   
4          0.0          0.0        0.0       38.50  ...       I       U   

   kid3_5  kid6_10  kid11_15  kid16_17  creditcd  eqpdays  Customer_ID  \
0       U        U         U        

####crclscod

In [8]:
crclscod_unicos = dataframe['crclscod'].unique() #Obtenemos todos los valores unicos de la columna 'crclscod'

#Mostramos la lista de valores unicos y la cantidad de estos
print(crclscod_unicos)
print(len(crclscod_unicos))

['A' 'EA' 'C' 'B' 'BA' 'D' 'G' 'CA' 'I' 'AA' 'J' 'U' 'Z' 'K' 'E' 'E4' 'W'
 'DA' 'Y' 'CC' 'GA' 'D4' 'D5' 'H' 'ZA' 'JF' 'B2' 'IF' 'Z1' 'O' 'M' 'L'
 'U1' 'Z4' 'A2' 'V' 'TP' 'EM' 'C2' 'C5' 'S' 'EC' 'Z5' 'D2' 'GY' 'ZY' 'E2'
 'A3' 'V1' 'EF' 'Z2' 'CY' 'P1' 'ZF']
54


In [9]:
#asignamos valores numericos equivalentes
crclscod_cat = dataframe[['crclscod']]
dataframe['crclscod_ordinal'] = ordinal_encoder.fit_transform(crclscod_cat)


#Quitamos la columna 'crclscod'
dataframe.drop('crclscod', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   rev_Mean  mou_Mean  totmrc_Mean  da_Mean  ovrmou_Mean  ovrrev_Mean  \
0   23.9975    219.25       22.500   0.2475         0.00          0.0   
1   57.4925    482.75       37.425   0.2475        22.75          9.1   
2   16.9900     10.25       16.990   0.0000         0.00          0.0   
3   38.0000      7.50       38.000   0.0000         0.00          0.0   
4   55.2300    570.50       71.980   0.0000         0.00          0.0   

   vceovr_Mean  datovr_Mean  roam_Mean  change_mou  ...  kid0_2  kid3_5  \
0          0.0          0.0        0.0     -157.25  ...       U       U   
1          9.1          0.0        0.0      532.25  ...       U       U   
2          0.0          0.0        0.0       -4.25  ...       U       Y   
3          0.0          0.0        0.0       -1.50  ...       Y       U   
4          0.0          0.0        0.0       38.50  ...       U       U   

   kid6_10  kid11_15  kid16_17  creditcd  eqpdays  Customer_ID  \
0        U         U         U         Y    

In [10]:
print(dataframe['crclscod_ordinal'])

0         0.0
1        21.0
2         7.0
3         4.0
4         0.0
         ... 
99995     4.0
99996    12.0
99997    17.0
99998    21.0
99999     4.0
Name: crclscod_ordinal, Length: 100000, dtype: float64


####asl_flag

In [11]:
asl_flag_unicos = dataframe['asl_flag'].unique() #Obtenemos todos los valores unicos de la columna 'asl_flag'

#Mostramos la lista de valores unicos y la cantidad de estos
print(asl_flag_unicos)
print(len(asl_flag_unicos))

['N' 'Y']
2


In [12]:
#Se asigna valores numericos a cada uno de los valores unicos
asl_flag_ordinal_map = {
    'Y': 1,
    'N': 2,
}

# Se aplica el mapeo a la columna 'asl_flag' y se crea una nueva columna 'asl_flag_ordinal' con los valores numericos respectivos
dataframe['asl_flag_ordinal'] = dataframe['asl_flag'].map(asl_flag_ordinal_map)

#Quitamos la columna 'asl_flag'
dataframe.drop('asl_flag', axis=1, inplace=True)

#verificamos
#print(dataframe.head())
print(dataframe['asl_flag_ordinal'])

0        2
1        2
2        2
3        2
4        2
        ..
99995    2
99996    1
99997    2
99998    2
99999    2
Name: asl_flag_ordinal, Length: 100000, dtype: int64


####prizm_social_one


In [13]:
prizm_social_one_unicos = dataframe['prizm_social_one'].unique() #Obtenemos todos los valores unicos de la columna 'prizm_social_one'

#Mostramos la lista de valores unicos y la cantidad de estos
print(prizm_social_one_unicos)
print(len(prizm_social_one_unicos))

['S' 'U' 'T' 'C' nan 'R']
6


In [14]:
#asignamos valores numericos equivalentes
prizm_social_one_cat = dataframe[['prizm_social_one']]
dataframe['prizm_social_one_ordinal'] = ordinal_encoder.fit_transform(prizm_social_one_cat)


#Quitamos la columna 'prizm_social_one'
dataframe.drop('prizm_social_one', axis=1, inplace=True)

#verificamos
#print(dataframe.head())
print(dataframe['prizm_social_one_ordinal'])

0        2.0
1        4.0
2        2.0
3        3.0
4        4.0
        ... 
99995    4.0
99996    2.0
99997    4.0
99998    4.0
99999    2.0
Name: prizm_social_one_ordinal, Length: 100000, dtype: float64


####area

In [15]:
area_unicos = dataframe['area'].unique() #Obtenemos todos los valores unicos de la columna 'area'

#Mostramos la lista de valores unicos y la cantidad de estos
print(area_unicos)
print(len(area_unicos))

['NORTHWEST/ROCKY MOUNTAIN AREA' 'CHICAGO AREA' 'GREAT LAKES AREA'
 'NEW ENGLAND AREA' 'DALLAS AREA' 'CENTRAL/SOUTH TEXAS AREA'
 'TENNESSEE AREA' 'MIDWEST AREA' 'PHILADELPHIA AREA' 'OHIO AREA'
 'HOUSTON AREA' 'SOUTHWEST AREA' 'NEW YORK CITY AREA'
 'ATLANTIC SOUTH AREA' 'SOUTH FLORIDA AREA' 'CALIFORNIA NORTH AREA'
 'DC/MARYLAND/VIRGINIA AREA' 'NORTH FLORIDA AREA' nan 'LOS ANGELES AREA']
20


In [16]:
#asignamos valores numericos equivalentes
area_cat = dataframe[['area']]
dataframe['area_ordinal'] = ordinal_encoder.fit_transform(area_cat)


#Quitamos la columna 'area'
dataframe.drop('area', axis=1, inplace=True)

#verificamos
#print(dataframe.head())
print(dataframe['area_ordinal'])

0        13.0
1         3.0
2         6.0
3         3.0
4        10.0
         ... 
99995     8.0
99996     8.0
99997    17.0
99998    17.0
99999     8.0
Name: area_ordinal, Length: 100000, dtype: float64


####dualband

In [17]:
dualband_unicos = dataframe['dualband'].unique() #Obtenemos todos los valores unicos de la columna 'dualband'

#Mostramos la lista de valores unicos y la cantidad de estos
print(dualband_unicos)
print(len(dualband_unicos))

['Y' 'N' 'T' 'U' nan]
5


In [18]:
#asignamos valores numericos equivalentes
dualband_cat = dataframe[['dualband']]
dataframe['dualband_ordinal'] = ordinal_encoder.fit_transform(dualband_cat)


#Quitamos la columna 'dualband'
dataframe.drop('dualband', axis=1, inplace=True)

#verificamos
#print(dataframe.head())
print(dataframe['dualband_ordinal'])

0        3.0
1        0.0
2        0.0
3        0.0
4        3.0
        ... 
99995    0.0
99996    0.0
99997    3.0
99998    3.0
99999    3.0
Name: dualband_ordinal, Length: 100000, dtype: float64


####refurb_new

In [19]:
refurb_new_unicos = dataframe['refurb_new'].unique() #Obtenemos todos los valores unicos de la columna 'refurb_new'

#Mostramos la lista de valores unicos y la cantidad de estos
print(refurb_new_unicos)
print(len(refurb_new_unicos))

['N' 'R' nan]
3


In [20]:
#asignamos valores numericos equivalentes
refurb_new_cat = dataframe[['refurb_new']]
dataframe['refurb_new_ordinal'] = ordinal_encoder.fit_transform(refurb_new_cat)


#Quitamos la columna 'refurb_new'
dataframe.drop('refurb_new', axis=1, inplace=True)

#verificamos
#print(dataframe.head())
print(dataframe['refurb_new_ordinal'])

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
99995    0.0
99996    0.0
99997    0.0
99998    0.0
99999    0.0
Name: refurb_new_ordinal, Length: 100000, dtype: float64


####hnd_webcap

In [21]:
hnd_webcap_unicos = dataframe['hnd_webcap'].unique() #Obtenemos todos los valores unicos de la columna 'hnd_webcap'

#Mostramos la lista de valores unicos y la cantidad de estos
print(hnd_webcap_unicos)
print(len(hnd_webcap_unicos))

['WCMB' 'WC' nan 'UNKW']
4


In [22]:
#asignamos valores numericos equivalentes
hnd_webcap_cat = dataframe[['hnd_webcap']]
dataframe['hnd_webcap_ordinal'] = ordinal_encoder.fit_transform(hnd_webcap_cat)


#Quitamos la columna 'hnd_webcap'
dataframe.drop('hnd_webcap', axis=1, inplace=True)

#verificamos
#print(dataframe.head())
print(dataframe['hnd_webcap_ordinal'])

0        2.0
1        1.0
2        NaN
3        NaN
4        2.0
        ... 
99995    1.0
99996    1.0
99997    2.0
99998    2.0
99999    2.0
Name: hnd_webcap_ordinal, Length: 100000, dtype: float64


####ownrent

En este caso se puedo observar que esta caracteristica contiene una gran cantidad de valores **nan**, por lo que directamente de eliminara la columna.

In [23]:
ownrent_unicos = dataframe['ownrent'].unique() #Obtenemos todos los valores unicos de la columna 'ownrent'

#Mostramos la lista de valores unicos y la cantidad de estos
print(ownrent_unicos)
print(len(ownrent_unicos))

['O' nan 'R']
3


In [24]:
#Quitamos la columna 'ownrent'
dataframe.drop('ownrent', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   rev_Mean  mou_Mean  totmrc_Mean  da_Mean  ovrmou_Mean  ovrrev_Mean  \
0   23.9975    219.25       22.500   0.2475         0.00          0.0   
1   57.4925    482.75       37.425   0.2475        22.75          9.1   
2   16.9900     10.25       16.990   0.0000         0.00          0.0   
3   38.0000      7.50       38.000   0.0000         0.00          0.0   
4   55.2300    570.50       71.980   0.0000         0.00          0.0   

   vceovr_Mean  datovr_Mean  roam_Mean  change_mou  ...  eqpdays  Customer_ID  \
0          0.0          0.0        0.0     -157.25  ...    361.0      1000001   
1          9.1          0.0        0.0      532.25  ...    240.0      1000002   
2          0.0          0.0        0.0       -4.25  ...   1504.0      1000003   
3          0.0          0.0        0.0       -1.50  ...   1812.0      1000004   
4          0.0          0.0        0.0       38.50  ...    434.0      1000005   

   new_cell_ordinal  crclscod_ordinal  asl_flag_ordinal  \
0              

####dwlltype

En este caso se puedo observar que esta caracteristica contiene una gran cantidad de valores **nan**, por lo que directamente de eliminara la columna.

In [25]:
dwlltype_unicos = dataframe['dwlltype'].unique() #Obtenemos todos los valores unicos de la columna 'dwlltype'

#Mostramos la lista de valores unicos y la cantidad de estos
print(dwlltype_unicos)
print(len(dwlltype_unicos))

['S' 'M' nan]
3


In [26]:
#Quitamos la columna 'dwlltype'
dataframe.drop('dwlltype', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   rev_Mean  mou_Mean  totmrc_Mean  da_Mean  ovrmou_Mean  ovrrev_Mean  \
0   23.9975    219.25       22.500   0.2475         0.00          0.0   
1   57.4925    482.75       37.425   0.2475        22.75          9.1   
2   16.9900     10.25       16.990   0.0000         0.00          0.0   
3   38.0000      7.50       38.000   0.0000         0.00          0.0   
4   55.2300    570.50       71.980   0.0000         0.00          0.0   

   vceovr_Mean  datovr_Mean  roam_Mean  change_mou  ...  eqpdays  Customer_ID  \
0          0.0          0.0        0.0     -157.25  ...    361.0      1000001   
1          9.1          0.0        0.0      532.25  ...    240.0      1000002   
2          0.0          0.0        0.0       -4.25  ...   1504.0      1000003   
3          0.0          0.0        0.0       -1.50  ...   1812.0      1000004   
4          0.0          0.0        0.0       38.50  ...    434.0      1000005   

   new_cell_ordinal  crclscod_ordinal  asl_flag_ordinal  \
0              

####marital

In [27]:
marital_unicos = dataframe['marital'].unique() #Obtenemos todos los valores unicos de la columna 'marital'

#Mostramos la lista de valores unicos y la cantidad de estos
print(marital_unicos)
print(len(marital_unicos))

['S' 'M' 'A' 'U' 'B' nan]
6


In [28]:
#asignamos valores numericos equivalentes
marital_cat = dataframe[['marital']]
dataframe['marital_ordinal'] = ordinal_encoder.fit_transform(marital_cat)


#Quitamos la columna 'marital'
dataframe.drop('marital', axis=1, inplace=True)

#verificamos
#print(dataframe.head())
print(dataframe['marital_ordinal'])

0        3.0
1        3.0
2        2.0
3        2.0
4        3.0
        ... 
99995    2.0
99996    2.0
99997    1.0
99998    4.0
99999    0.0
Name: marital_ordinal, Length: 100000, dtype: float64


####infobase

En este caso se puedo observar que esta caracteristica contiene una gran cantidad de valores **nan**, por lo que directamente de eliminara la columna.

In [29]:
infobase_unicos = dataframe['infobase'].unique() #Obtenemos todos los valores unicos de la columna 'infobase'

#Mostramos la lista de valores unicos y la cantidad de estos
print(infobase_unicos)
print(len(infobase_unicos))

['M' nan 'N']
3


In [30]:
#Quitamos la columna 'infobase'
dataframe.drop('infobase', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   rev_Mean  mou_Mean  totmrc_Mean  da_Mean  ovrmou_Mean  ovrrev_Mean  \
0   23.9975    219.25       22.500   0.2475         0.00          0.0   
1   57.4925    482.75       37.425   0.2475        22.75          9.1   
2   16.9900     10.25       16.990   0.0000         0.00          0.0   
3   38.0000      7.50       38.000   0.0000         0.00          0.0   
4   55.2300    570.50       71.980   0.0000         0.00          0.0   

   vceovr_Mean  datovr_Mean  roam_Mean  change_mou  ...  Customer_ID  \
0          0.0          0.0        0.0     -157.25  ...      1000001   
1          9.1          0.0        0.0      532.25  ...      1000002   
2          0.0          0.0        0.0       -4.25  ...      1000003   
3          0.0          0.0        0.0       -1.50  ...      1000004   
4          0.0          0.0        0.0       38.50  ...      1000005   

   new_cell_ordinal  crclscod_ordinal  asl_flag_ordinal  \
0                 1               0.0                 2   
1         

####HHstatin

En este caso se puedo observar que esta caracteristica contiene una gran cantidad de valores **nan**, por lo que directamente de eliminara la columna.

In [31]:
HHstatin_unicos = dataframe['HHstatin'].unique() #Obtenemos todos los valores unicos de la columna 'HHstatin'

#Mostramos la lista de valores unicos y la cantidad de estos
print(HHstatin_unicos)
print(len(HHstatin_unicos))

['C' 'I' nan 'B' 'A' 'G' 'H']
7


In [32]:
#Quitamos la columna 'HHstatin'
dataframe.drop('HHstatin', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   rev_Mean  mou_Mean  totmrc_Mean  da_Mean  ovrmou_Mean  ovrrev_Mean  \
0   23.9975    219.25       22.500   0.2475         0.00          0.0   
1   57.4925    482.75       37.425   0.2475        22.75          9.1   
2   16.9900     10.25       16.990   0.0000         0.00          0.0   
3   38.0000      7.50       38.000   0.0000         0.00          0.0   
4   55.2300    570.50       71.980   0.0000         0.00          0.0   

   vceovr_Mean  datovr_Mean  roam_Mean  change_mou  ...  Customer_ID  \
0          0.0          0.0        0.0     -157.25  ...      1000001   
1          9.1          0.0        0.0      532.25  ...      1000002   
2          0.0          0.0        0.0       -4.25  ...      1000003   
3          0.0          0.0        0.0       -1.50  ...      1000004   
4          0.0          0.0        0.0       38.50  ...      1000005   

   new_cell_ordinal  crclscod_ordinal  asl_flag_ordinal  \
0                 1               0.0                 2   
1         

####dwllsize

En este caso se puedo observar que esta caracteristica contiene una gran cantidad de valores **nan**, por lo que directamente de eliminara la columna.

In [33]:
dwllsize_unicos = dataframe['dwllsize'].unique() #Obtenemos todos los valores unicos de la columna 'dwllsize'

#Mostramos la lista de valores unicos y la cantidad de estos
print(dwllsize_unicos)
print(len(dwllsize_unicos))

['A' 'D' 'O' nan 'E' 'J' 'C' 'B' 'I' 'N' 'K' 'L' 'G' 'H' 'M' 'F']
16


In [34]:
#Quitamos la columna 'dwllsize'
dataframe.drop('dwllsize', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   rev_Mean  mou_Mean  totmrc_Mean  da_Mean  ovrmou_Mean  ovrrev_Mean  \
0   23.9975    219.25       22.500   0.2475         0.00          0.0   
1   57.4925    482.75       37.425   0.2475        22.75          9.1   
2   16.9900     10.25       16.990   0.0000         0.00          0.0   
3   38.0000      7.50       38.000   0.0000         0.00          0.0   
4   55.2300    570.50       71.980   0.0000         0.00          0.0   

   vceovr_Mean  datovr_Mean  roam_Mean  change_mou  ...  Customer_ID  \
0          0.0          0.0        0.0     -157.25  ...      1000001   
1          9.1          0.0        0.0      532.25  ...      1000002   
2          0.0          0.0        0.0       -4.25  ...      1000003   
3          0.0          0.0        0.0       -1.50  ...      1000004   
4          0.0          0.0        0.0       38.50  ...      1000005   

   new_cell_ordinal  crclscod_ordinal  asl_flag_ordinal  \
0                 1               0.0                 2   
1         

####ethnic

In [35]:
ethnic_unicos = dataframe['ethnic'].unique() #Obtenemos todos los valores unicos de la columna 'ethnic'

#Mostramos la lista de valores unicos y la cantidad de estos
print(ethnic_unicos)
print(len(ethnic_unicos))

['N' 'Z' 'U' 'I' 'S' 'F' 'J' 'M' 'H' 'G' 'D' 'O' nan 'R' 'B' 'P' 'X' 'C']
18


In [36]:
#asignamos valores numericos equivalentes
ethnic_cat = dataframe[['ethnic']]
dataframe['ethnic_ordinal'] = ordinal_encoder.fit_transform(ethnic_cat)


#Quitamos la columna 'ethnic'
dataframe.drop('ethnic', axis=1, inplace=True)

#verificamos
#print(dataframe.head())
print(dataframe['ethnic_ordinal'])

0         9.0
1        16.0
2         9.0
3        14.0
4         6.0
         ... 
99995    13.0
99996     9.0
99997    14.0
99998    13.0
99999     5.0
Name: ethnic_ordinal, Length: 100000, dtype: float64


####kid0_2

In [37]:
kid0_2_unicos = dataframe['kid0_2'].unique() #Obtenemos todos los valores unicos de la columna 'kid0_2'

#Mostramos la lista de valores unicos y la cantidad de estos
print(kid0_2_unicos)
print(len(kid0_2_unicos))

['U' 'Y' nan]
3


In [38]:
#asignamos valores numericos equivalentes
kid0_2_cat = dataframe[['kid0_2']]
dataframe['kid0_2_ordinal'] = ordinal_encoder.fit_transform(kid0_2_cat)


#Quitamos la columna 'ethnic'
dataframe.drop('kid0_2', axis=1, inplace=True)

#verificamos
#print(dataframe.head())
print(dataframe['kid0_2_ordinal'])

0        0.0
1        0.0
2        0.0
3        1.0
4        0.0
        ... 
99995    0.0
99996    0.0
99997    1.0
99998    0.0
99999    0.0
Name: kid0_2_ordinal, Length: 100000, dtype: float64


####kid3_5

In [39]:
kid3_5_unicos = dataframe['kid3_5'].unique() #Obtenemos todos los valores unicos de la columna 'kid3_5'

#Mostramos la lista de valores unicos y la cantidad de estos
print(kid3_5_unicos)
print(len(kid3_5_unicos))

['U' 'Y' nan]
3


In [40]:
#asignamos valores numericos equivalentes
kid3_5_cat = dataframe[['kid3_5']]
dataframe['kid3_5_ordinal'] = ordinal_encoder.fit_transform(kid3_5_cat)


#Quitamos la columna 'ethnic'
dataframe.drop('kid3_5', axis=1, inplace=True)

#verificamos
#print(dataframe.head())
print(dataframe['kid3_5_ordinal'])

0        0.0
1        0.0
2        1.0
3        0.0
4        0.0
        ... 
99995    0.0
99996    0.0
99997    1.0
99998    0.0
99999    0.0
Name: kid3_5_ordinal, Length: 100000, dtype: float64


####kid6_10

In [41]:
kid6_10_unicos = dataframe['kid6_10'].unique() #Obtenemos todos los valores unicos de la columna 'kid6_10'

#Mostramos la lista de valores unicos y la cantidad de estos
print(kid6_10_unicos)
print(len(kid6_10_unicos))

['U' 'Y' nan]
3


In [42]:
#asignamos valores numericos equivalentes
kid6_10_cat = dataframe[['kid6_10']]
dataframe['kid6_10_ordinal'] = ordinal_encoder.fit_transform(kid6_10_cat)


#Quitamos la columna 'kid6_10'
dataframe.drop('kid6_10', axis=1, inplace=True)

#verificamos
#print(dataframe.head())
print(dataframe['kid6_10_ordinal'])

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
99995    0.0
99996    1.0
99997    0.0
99998    0.0
99999    0.0
Name: kid6_10_ordinal, Length: 100000, dtype: float64


####kid11_15

In [43]:
kid11_15_unicos = dataframe['kid11_15'].unique() #Obtenemos todos los valores unicos de la columna 'kid11_15'

#Mostramos la lista de valores unicos y la cantidad de estos
print(kid11_15_unicos)
print(len(kid11_15_unicos))

['U' 'Y' nan]
3


In [44]:
#asignamos valores numericos equivalentes
kid11_15_cat = dataframe[['kid11_15']]
dataframe['kid11_15_ordinal'] = ordinal_encoder.fit_transform(kid11_15_cat)


#Quitamos la columna 'kid11_15'
dataframe.drop('kid11_15', axis=1, inplace=True)

#verificamos
#print(dataframe.head())
print(dataframe['kid11_15_ordinal'])

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
99995    1.0
99996    1.0
99997    0.0
99998    0.0
99999    0.0
Name: kid11_15_ordinal, Length: 100000, dtype: float64


####kid16_17

In [45]:
kid16_17_unicos = dataframe['kid16_17'].unique() #Obtenemos todos los valores unicos de la columna 'kid16_17'

#Mostramos la lista de valores unicos y la cantidad de estos
print(kid16_17_unicos)
print(len(kid16_17_unicos))

['U' 'Y' nan]
3


In [46]:
#asignamos valores numericos equivalentes
kid16_17_cat = dataframe[['kid16_17']]
dataframe['kid16_17_ordinal'] = ordinal_encoder.fit_transform(kid16_17_cat)


#Quitamos la columna 'kid16_17'
dataframe.drop('kid16_17', axis=1, inplace=True)

#verificamos
#print(dataframe.head())
print(dataframe['kid16_17_ordinal'])

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
99995    0.0
99996    1.0
99997    0.0
99998    0.0
99999    0.0
Name: kid16_17_ordinal, Length: 100000, dtype: float64


####creditcd

In [47]:
creditcd_unicos = dataframe['creditcd'].unique() #Obtenemos todos los valores unicos de la columna 'creditcd'

#Mostramos la lista de valores unicos y la cantidad de estos
print(creditcd_unicos)
print(len(creditcd_unicos))

['Y' 'N' nan]
3


In [48]:
#asignamos valores numericos equivalentes
creditcd_cat = dataframe[['creditcd']]
dataframe['creditcd_ordinal'] = ordinal_encoder.fit_transform(creditcd_cat)


#Quitamos la columna 'creditcd'
dataframe.drop('creditcd', axis=1, inplace=True)

#verificamos
#print(dataframe.head())
print(dataframe['creditcd_ordinal'])

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
99995    1.0
99996    1.0
99997    0.0
99998    0.0
99999    0.0
Name: creditcd_ordinal, Length: 100000, dtype: float64


###Correlacion

La correlación es una medida estadística que indica el grado de relación entre las variables. Necesitamos identificar:

* Variables independientes con una correlacion muy fuerte entre si
* Variables independientes con una correlacion muy debil con la variable dependiente

Estos deben ser eliminados ya que pueden afectar negativamente al entrenamiento del modelo

In [49]:
correlation_matrix = dataframe.corr()

# Imprime la matriz de correlación
print(correlation_matrix)

                  rev_Mean  mou_Mean  totmrc_Mean   da_Mean  ovrmou_Mean  \
rev_Mean          1.000000  0.706029     0.602148  0.403806     0.772117   
mou_Mean          0.706029  1.000000     0.575567  0.394583     0.575611   
totmrc_Mean       0.602148  0.575567     1.000000  0.306966     0.200732   
da_Mean           0.403806  0.394583     0.306966  1.000000     0.304596   
ovrmou_Mean       0.772117  0.575611     0.200732  0.304596     1.000000   
...                    ...       ...          ...       ...          ...   
kid3_5_ordinal   -0.013761 -0.030699    -0.014298 -0.004224    -0.007247   
kid6_10_ordinal  -0.026651 -0.047100    -0.029562 -0.012542    -0.020530   
kid11_15_ordinal -0.030874 -0.040724    -0.038707 -0.021704    -0.019049   
kid16_17_ordinal -0.023279 -0.010262    -0.029823 -0.024409    -0.011931   
creditcd_ordinal -0.079366 -0.125219    -0.087584 -0.042182    -0.048028   

                  ovrrev_Mean  vceovr_Mean  datovr_Mean  roam_Mean  \
rev_Mean         

###Verificamos si hay nulos

In [50]:
print(dataframe.isnull().sum())

rev_Mean             357
mou_Mean             357
totmrc_Mean          357
da_Mean              357
ovrmou_Mean          357
                    ... 
kid3_5_ordinal      1732
kid6_10_ordinal     1732
kid11_15_ordinal    1732
kid16_17_ordinal    1732
creditcd_ordinal    1732
Length: 95, dtype: int64


Eliminamos todas las filas que contengan valores nulos

In [51]:
dataframe.dropna(inplace=True)

In [52]:
print(dataframe.isnull().sum())

rev_Mean            0
mou_Mean            0
totmrc_Mean         0
da_Mean             0
ovrmou_Mean         0
                   ..
kid3_5_ordinal      0
kid6_10_ordinal     0
kid11_15_ordinal    0
kid16_17_ordinal    0
creditcd_ordinal    0
Length: 95, dtype: int64


###Dividimos el dataframe para el entrenamiento y la prueba



Dividimos el dataframe entre las variables independientes y la variable dependiente

In [53]:
y = dataframe['churn']
X = dataframe.drop(columns=['churn'])

In [54]:
print(X.head())
print(y.head())

    rev_Mean  mou_Mean  totmrc_Mean  da_Mean  ovrmou_Mean  ovrrev_Mean  \
0    23.9975    219.25        22.50   0.2475          0.0          0.0   
4    55.2300    570.50        71.98   0.0000          0.0          0.0   
5    82.2750   1312.25        75.00   1.2375          0.0          0.0   
9    31.6625     25.50        29.99   0.2475          0.0          0.0   
14  212.5150   1971.50        84.99   2.2275        249.5         99.8   

    vceovr_Mean  datovr_Mean  roam_Mean  change_mou  ...  refurb_new_ordinal  \
0           0.0          0.0     0.0000     -157.25  ...                 0.0   
4           0.0          0.0     0.0000       38.50  ...                 0.0   
5           0.0          0.0     0.0000      156.75  ...                 0.0   
9           0.0          0.0     0.0000       59.50  ...                 0.0   
14         99.8          0.0    35.4975     -200.50  ...                 0.0   

    hnd_webcap_ordinal  marital_ordinal  ethnic_ordinal  kid0_2_ordinal  \

Dividimos los datos en conjuntos para entrenamiento y para prueba, indicando que se usara el 80% para entrenar y el resto para probar.

In [55]:
# Dividimos los datos en conjuntos para entrenamiento y para prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [56]:
print(len(X_train))
print(len(y_train))

31676
31676


Pasamos los datos del DataFrame a un array de NumPy


In [57]:
X_trainNp = X_train.values
y_trainNp= y_train.values

In [58]:
X_testNp = X_test.values
y_testNp= y_test.values

In [59]:
print(X_trainNp.shape[1])#cantidad de caracteristicas

94


##Red Neuronal

Definimos la clase **RedNeuronalMLS** que hereda de **nn.Module** que es la clase base para todos los modulos de red neuronal de PyTorch.

Esta clase se definen las capas de la red neuronal y el metodo forward en donde se define el comportamiento de propagacion hacia adelante

In [60]:
class RedNeuronalMLS(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RedNeuronalMLS, self).__init__() #llama al constructor de la clase nn.Module

        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):

        x = self.fc1(x) #los datos pasan a traves de la capa oculta
        x = F.relu(x) #se aplica la funcion de activacion ReLu
        x = self.fc2(x) # la salida de la capa oculta pasa a la capa de salida
        x = torch.sigmoid(x) #se aplica la funcion sigmoide para obtener valores entre 0 y 1

        return x

Establecemos el dispositivo en el que se ejecutara la red neuronal y los valores de los hiperparametros

In [61]:
# Si una GPU con CUDA esta disponible se la utilizara para hacer los calculos, si no se utilizara un CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparametros
input_size = 94 #cantidad de caracteristicas
num_classes = 1 #clases de salida
learning_rate = 0.01 #tasa de aprendizaje
batch_size = 512 #tamaño del lote
num_epochs = 1000 #numero de iteraciones

cuda


Definimos la funcion **featureNormalize** para normalizar las caracteristicas. Esto convierte los valores para que esten en el rango de -1 y 1. La normalizacion se realiza para que el aprendizaje del modelo converja mas rapido al entrenar.

In [62]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)#calcula la media de cada columna
    sigma = np.std(X, axis = 0)#calcula la desviacion estandar de cada columna
    X_norm = (X - mu) / sigma #normaliza la matriz X (-1, 1)

    return X_norm

Normalizamos las caracteristicas de entrenamiento y de prueba

In [63]:
# llama featureNormalize con los datos cargados
X_NormtrainNp = featureNormalize(X_trainNp)
#se normaliza las caracteristicas de prueba
X_NormtestNp = featureNormalize(X_testNp)

Convertimos los arrays de NumPy a tensores de PyTorch, que es el tipo de dato basico utilizado en PyTorch para almacenar y operar con los datos.
Tambien creamos DataLoaders para los conjutnos de datos de entrenamiento y prueba para poder iterar sobre los lotes.

In [64]:
#conviertiendo a tensores de PyTorch
X_train = torch.from_numpy(X_NormtrainNp).float()
y_train = torch.from_numpy(y_trainNp).float()
X_test = torch.from_numpy(X_NormtestNp).float()
y_test = torch.from_numpy(y_testNp).float()
#Creando los conjuntos de datos
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
#creando cargadores de datos
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Instanciamos nuestro modelo

In [65]:
# Initialize network
model = RedNeuronalMLS(input_size=input_size, num_classes=num_classes).to(device)

Observamos las capas y nodos de nuestro modelo

In [66]:
model

RedNeuronalMLS(
  (fc1): Linear(in_features=94, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=1, bias=True)
)

Establecemos la funcion de perdida que en este caso sera la Binary Cross Entropy combinada con la funcion sigmoide.
Tambien definimos el optimizador para el descenso por el gradiente que sera Adam.

In [67]:
# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

###Entrenamiento de la red neuronal

In [68]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):

        # movemos los datos y las etiquetas al dispositivo (GPU o CPU)
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Redimensionamos los datos
        data = data.reshape(data.shape[0], -1)
        print(data.shape)

        # forward
        scores = model(data)
        scores = scores.view(-1)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad() #se resetean los gradientes acumulados
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

Output hidden; open in https://colab.research.google.com to view.

Definimos la funcion para calcular la presicion de la red neuronal. Posteriormente la utilizamos con los datos de entrenamiento y los datos de prueba.

In [69]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

p_train, pred_train  = check_accuracy(train_loader, model)
p_test, pred_test  = check_accuracy(test_loader, model)

print(f"Accuracy on training set: {p_train*100:.2f}")
print(f"Accuracy on test set: {p_test*100:.2f}")

Accuracy on training set: 52.47
Accuracy on test set: 52.32


El cambio de los hiperparametros no mostro ninguna inluencia en el resultado.